DEPENDENCIAS NECESARIAS


In [13]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-auth-oauthlib[youtube]
!pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install sumy textract nltk pandas numpy yfinance requests
!pip install PyPDF2
!pip install python-docx PyPDF2 python-pptx




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


CÓDIGO DE PRUEBA

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
docs_credentials_path = '/content/drive/My Drive/Docsolver/Credentials Gdoc/Docs.json'
drive_credentials_path = '/content/drive/My Drive/Docsolver/Credentials Gdoc/Drive.json'


CÓDIGO FINAL

In [32]:
import os
import json
from googleapiclient.discovery import build
from google.oauth2 import service_account
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from google.colab import files
from PyPDF2 import PdfReader

# Cargar credenciales de Google Drive
drive_credentials_path = '/content/drive/My Drive/Docsolver/Credentials Gdoc/Drive.json'
with open(drive_credentials_path) as drive_file:
    drive_credentials = service_account.Credentials.from_service_account_info(json.load(drive_file))

# Construir el servicio de Google Drive
drive_service = build('drive', 'v3', credentials=drive_credentials)

# Crear un nuevo documento de Google Docs
document_metadata = {
    'name': 'Resumen',
    'mimeType': 'application/vnd.google-apps.document'
}
new_document = drive_service.files().create(body=document_metadata).execute()
document_id = new_document['id']

print(f'Documento creado: https://docs.google.com/document/d/{document_id}')

# Compartir el archivo con la cuenta de servicio de Google Docs y otorgar permisos de escritura
permission_docs = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'docsolver@docsolver.iam.gserviceaccount.com'  # Correo electrónico de la cuenta de servicio de Google Docs
}
drive_service.permissions().create(fileId=document_id, body=permission_docs).execute()

# Compartir el archivo con la cuenta de servicio de Google Drive y otorgar permisos de escritura
permission_drive = {
    'type': 'user',
    'role': 'writer',
    'emailAddress': 'docsolver-260@docsolver.iam.gserviceaccount.com'  # Correo electrónico de la cuenta de servicio de Google Drive
}
drive_service.permissions().create(fileId=document_id, body=permission_drive).execute()

# Construir el servicio de Google Docs
docs_credentials_path = '/content/drive/My Drive/Docsolver/Credentials Gdoc/Docs.json'
with open(docs_credentials_path) as docs_file:
    docs_credentials = service_account.Credentials.from_service_account_info(json.load(docs_file))

docs_service = build('docs', 'v1', credentials=docs_credentials)

# Permitir al usuario seleccionar un archivo localmente
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Leer el archivo PDF y extraer el texto
if filename.endswith('.pdf'):
    with open(filename, 'rb') as file:
        reader = PdfReader(file)
        number_of_pages = len(reader.pages)
        full_text = []
        for page in reader.pages:
            full_text.append(page.extract_text())

# Convertir la lista de texto en un solo string
content = '\n'.join(full_text)

# Generar el resumen utilizando Sumy
parser = PlaintextParser.from_string(content, Tokenizer('spanish'))
summarizer = LsaSummarizer()
summary = summarizer(parser.document, sentences_count=5)  # El resumen tendrá 5 oraciones

# Convertir el resumen a texto plano
summary_text = ' '.join(str(sentence) for sentence in summary)

# Escribir el resumen en el documento
document_content = {
    'requests': [
        {
            'insertText': {
                'location': {
                    'index': 1,
                },
                'text': summary_text + '\n' if summary_text else 'No se pudo generar el resumen.\n'
            }
        }
    ]
}
docs_service.documents().batchUpdate(documentId=document_id, body=document_content).execute()

print('Resumen escrito en el documento.')

# Obtener el enlace del documento
document_url = f'https://docs.google.com/document/d/{document_id}'

# Compartir el enlace del documento con el usuario final
permission = {
    'type': 'anyone',
    'role': 'reader'
}
drive_service.permissions().create(fileId=document_id, body=permission).execute()

print(f'Enlace del documento compartido: {document_url}')


Documento creado: https://docs.google.com/document/d/1LRJl4WOWgp4E1JI_Ip0RH8dMFcRMoTOLNKG_tJAZKkI


Saving Macroeconomia_1-2024.pdf to Macroeconomia_1-2024 (5).pdf
Resumen escrito en el documento.
Enlace del documento compartido: https://docs.google.com/document/d/1LRJl4WOWgp4E1JI_Ip0RH8dMFcRMoTOLNKG_tJAZKkI
